### Library Imports

In [1]:
import pandas as pd
import xarray as xr
import numpy as np

### Data imports

In [2]:
# Data info : 
nb_samples = 4 # 4 samples per year
filename = 'France_mix_test.xlsx'
#filename = 'france_2050_mix_all.xlsx'
#filename = 'elect_table.xlsx'

In [3]:
elec_sheets = {}

for sample in range(nb_samples) : 
    year_day_sample = pd.DataFrame(pd.read_excel(filename,index_col = 0,sheet_name = sample))
    # Save, from the get-go, the possible years (needs more work to be fail proof)
    years = year_day_sample['YEAR'].unique()
    day_samples = year_day_sample.groupby('YEAR')
    
    elec_sheets[sample] = {}

    # Split years from the sheets    
    for year,data in day_samples : 
        data.drop(columns = 'YEAR',inplace = True)
        # Store both years according to number of samples : 
        elec_sheets[sample][year] =data

#print('Samples within a year :',elec_sheets.keys())
#print('Sample of years ',elec_sheets[0].keys())    
#print(years)    
    #elec_sheets[sample] = pd.DataFrame(pd.read_excel(filename,index_col = 0,sheet_name = sample))
display(elec_sheets[0][2050].head(5))

#ds_W_2020 = pd.read_excel('elect_table.xlsx', index_col = 'Hr')
#display(ds_2020.head(5))

,Nucleaire,Eolien en mer,Eolien terrestre,Centrales gaz,Solaire,STEP,Hydro,Imports nets,Autre renouvelables,Batteries
Hours,,,,,,,,,,
0,0.024209,0.125322,0.410421,0.021090,0.0,0.012294,0.177061,0.212947,0.016632,0.000024
1,0.024200,0.125050,0.408251,0.020609,0.0,0.011314,0.175082,0.218844,0.016650,0.000002
2,0.024324,0.125474,0.408526,0.020256,0.0,0.010368,0.174122,0.220175,0.016755,0.000000
3,0.024279,0.125242,0.406190,0.019938,0.0,0.009902,0.172074,0.225666,0.016709,0.000000
4,0.024419,0.125652,0.407136,0.019800,0.0,0.009696,0.172905,0.223574,0.016818,0.000000


In [4]:
# List the dimensions
Hours =  elec_sheets[0][years[0]].index.tolist()

# Technologies : 
cols2keep = []
for sheet_id in elec_sheets.keys() :
    for year in years : # redundant, but usefull to make sure nothing is off according to years
        cols = elec_sheets[sheet_id][years[0]].columns.to_list()   
        cols2keep.extend(cols)
technologies = [x for x in set(cols2keep)]

# Adapt dataframes in case some technologies are missing at this timepoint : 
for sheet_id in elec_sheets.keys() :
    for year in years : 
        # Find missing columns
        cols_to_add = [x for x in technologies if x not in elec_sheets[sheet_id][year].columns]

        # Add the columns (if needed) : 
        for col in cols_to_add :
            print(col)
            elec_sheets[sheet_id][year][col] = 0

        # While there, also make sure if a column is partial, just add zeros
        elec_sheets[sheet_id][year] = elec_sheets[sheet_id][year].fillna(0)

        # And also : relabel according to the technologies list (to match when the xarray is built)
        elec_sheets[sheet_id][year]= elec_sheets[sheet_id][year][technologies]
        
        # Quick check (optional)
        #print(year)
        #display(elec_sheets[sheet_id][year].head(5))

# Number of snapshots available for the year
date2_year = [1]
for value in range(nb_samples): 
    date2_year.append(round((value+1)*365/nb_samples))

# Overview
print(Hours)
print(technologies)
print(date2_year)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
['Nucleaire', 'Batteries', 'Imports nets', 'STEP', 'Eolien en mer', 'Autre renouvelables', 'Eolien terrestre', 'Hydro', 'Centrales gaz', 'Solaire']
[1, 91, 182, 274, 365]


In [5]:
# list the dataframes that need to be uploaded to xarray
yearly_data = []

# First by year
for year in years : 
    # Then by sample number (season here)
    data = [elec_sheets[sheet_id][year] for sheet_id in range(nb_samples)]
    
    # Append at the end the first entry (coming full circle) 
    data.append(elec_sheets[0][year])
    yearly_data.append(data)

In [6]:
# Creating the xarray DataArray
elec_mixes = xr.DataArray(
    yearly_data,
    coords=[years,date2_year,Hours, technologies],
    dims=["year","Date", "Hour","technology"]
)

In [7]:
# Check if its is properly working : 
display(elec_mixes.sel(
    #technology='Centrales gaz', 
    year = 2050,
    Date = 365,
    Hour = 3,
))

<xarray.DataArray (technology: 10)>
array([0.02427893, 0.        , 0.22566633, 0.0099018 , 0.12524214,
       0.01670877, 0.40618962, 0.17207397, 0.01993844, 0.        ])
Coordinates:
    year        int64 2050
    Date        int32 365
    Hour        int32 3
  * technology  (technology) <U19 'Nucleaire' 'Batteries' ... 'Solaire'

In [8]:
# Export for next steps.
elec_mixes.to_netcdf(path = 'elec_mixes.nc')

In [9]:
# Reload and make sure everything is okay : 
#elec_mixes_test = xr.open_dataset('elec_mixes.nc')
#display(elec_mixes)

### Selecting an array / single value in a more conventional format (not mandatory to make it an array/scalar)

In [10]:
# If selecting from this notebook

# Make a selection 
test = elec_mixes.sel(
    technology='Centrales gaz', 
    year = 2050,
    Date = 365,
    Hour = 3,
)
# Turn it into a nice array, then values
#test = test.to_array()
test = test.values # *5+2

print(test)

0.019938440241727994


In [11]:
# If loading from the netcdf file : 

# Make a selection 
test = elec_mixes.sel(
    technology='Centrales gaz', 
    year = 2050,
    Date = 365,
    Hour = 3,
)
# Turn it into a nice array, then values
test = test.to_array()
test = test.values[0] # *5+2

print(test)

AttributeError: 'DataArray' object has no attribute 'to_array'